# Geo part

In [1]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         394 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.

In [1]:
from geopy.distance import distance
from geopy.distance import great_circle

import json, logging
from geopy.geocoders import Nominatim

def seperator(a,b):
    c = (b - a)/9
    return [round(a,7),round(a+c,7),round(a+c*2,7),round(a+c*3,7),round(a+c*4,7),round(a+c*5,7),round(a+c*6,7),round(a+c*7,7),round(a+c*8,7),round(b,7)]

def geo_point_creator(city):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(city)
    lat_list = seperator(float(location.raw['boundingbox'][0]),float(location.raw['boundingbox'][1]))
    lon_list = seperator(float(location.raw['boundingbox'][2]),float(location.raw['boundingbox'][3]))
    point_list = []
    for lat in lat_list:
        for lon in lon_list:
            point_list.append([city,lat,lon])
    df_city = pd.DataFrame(point_list, columns=['city','lat','lng'])
    point0 = [float(df_city.loc[0,['lat']]),float(df_city.loc[0,['lng']])]
    point1 = [float(df_city.loc[1,['lat']]),float(df_city.loc[1,['lng']])]
    point10 = [float(df_city.loc[10,['lat']]),float(df_city.loc[10,['lng']])]
    rad1 = int(distance(point0, point1).meters//2)
    rad2 = int(distance(point0, point10).meters//2) 
    radius = (rad1 + rad2)//2
    #print(point0,point1,point10,rad1,rad2,radius)
    df_city['radius'] = radius
    return df_city

# Data Cleaning

In [2]:
import pandas as pd
import numpy as np
import requests
import re

df = pd.read_csv('CITIES_12062020101058739.csv')

In [15]:
gdp_2018 = df[df['Year']==2018]
top5_gdpt_city = gdp_2018[gdp_2018['VAR']=='GDP_REAL_PPP'].sort_values(by='Value',ascending=False).reset_index().head()
for i,n in enumerate(top5_gdpt_city['METRO_ID'].to_list()):
    x = ''.join(re.findall(r'[A-Za-z]', n))
    top5_gdpt_city['Flags'][i] = top5_gdpt_city['Metropolitan areas'].to_list()[i].replace(' (Greater)','') + ', ' + x

gdpt = top5_gdpt_city['Flags'].to_list()
gdpt[2] = 'Seoul'
gdpt

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


['New York, USA', 'Los Angeles, USA', 'Seoul', 'Washington, USA', 'Paris, FR']

In [16]:
top5_gdpp_city = gdp_2018[gdp_2018['VAR']=='GDP_PC_REAL_PPP'].sort_values(by='Value',ascending=False).reset_index().head()
for i,n in enumerate(top5_gdpp_city['METRO_ID'].to_list()):
    x = ''.join(re.findall(r'[A-Za-z]', n))
    top5_gdpp_city['Flags'][i] = top5_gdpp_city['Metropolitan areas'].to_list()[i].replace(' (Greater)','') + ', ' + x
gdpp = top5_gdpp_city['Flags'].to_list()
gdpp

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


['Cork, IE',
 'San Francisco, USA',
 'Durham, USA',
 'Luxembourg, LU',
 'Washington, USA']

In [3]:
CLIENT_ID = 'GBHOGHOE3DL1WYFOB4W5WI5SM1PLH1MDY4WN3O12SFGOVDUU' # your Foursquare ID
CLIENT_SECRET = 'AI25GODMBYEC331GD1VSY4Y3MRBP3SKR15N2YC01VKLSR0U4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100

Your credentails:
CLIENT_ID: GBHOGHOE3DL1WYFOB4W5WI5SM1PLH1MDY4WN3O12SFGOVDUU
CLIENT_SECRET:AI25GODMBYEC331GD1VSY4Y3MRBP3SKR15N2YC01VKLSR0U4


In [4]:
def getNearbyVenues(names, latitudes, longitudes, radiuses):
    
    venues_list=[]
    for name, lat, lng, radius in zip(names, latitudes, longitudes, radiuses):
        #print('{},{},{}'.format(lat,lng,radius))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City Name',
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**for loop to generate the DataFrame**

In [18]:
city_list = gdpt

In [22]:
city_point_gdpt = pd.DataFrame(columns=['city','lat','lng','radius'])
for city in city_list:
    df_tmp = geo_point_creator(city)
    city_point_gdpt = city_point_gdpt.append(df_tmp, ignore_index=True)
NY_point = city_point_gdpt[city_point_gdpt['city']=='New York, USA']
LA_point = city_point_gdpt[city_point_gdpt['city']=='Los Angeles, USA']
Seoul_point = city_point_gdpt[city_point_gdpt['city']=='Seoul']
DC_point = city_point_gdpt[city_point_gdpt['city']=='Washington, USA']
Paris_point = city_point_gdpt[city_point_gdpt['city']=='Paris, FR']

In [26]:
city_list = gdpp
city_point_gdpp = pd.DataFrame(columns=['city','lat','lng','radius'])
for city in city_list:
    df_tmp = geo_point_creator(city)
    city_point_gdpp = city_point_gdpp.append(df_tmp, ignore_index=True)

In [54]:
Durham_point = city_point_gdpp[city_point_gdpp['city']=='Durham, USA']
Luxembourg_point = city_point_gdpp[city_point_gdpp['city']=='Luxembourg, LU']
SF_point = city_point_gdpp[city_point_gdpp['city']=='San Francisco, USA']
DC_point = city_point_gdpp[city_point_gdpp['city']=='Washington, USA']
Cork_point = city_point_gdpp[city_point_gdpp['city']=='Cork, IE']

In [55]:
city_point = pd.DataFrame(columns=['city','lat','lng','radius'])
city_point = city_point.append(NY_point, ignore_index=True)
city_point = city_point.append(LA_point, ignore_index=True)
city_point = city_point.append(DC_point, ignore_index=True)
city_point = city_point.append(SF_point, ignore_index=True)
city_point = city_point.append(Paris_point, ignore_index=True)


In [56]:
city_point.shape

(500, 4)

In [56]:
SF_point.head()

,city,lat,lng,radius
100,"San Francisco, USA",37.640314,-123.173825,3080
101,"San Francisco, USA",37.640314,-123.074675,3080
102,"San Francisco, USA",37.640314,-122.975526,3080
103,"San Francisco, USA",37.640314,-122.876376,3080
104,"San Francisco, USA",37.640314,-122.777227,3080


# Get Venues and Create features

In [ ]:
NY_venues = getNearbyVenues(names=NY_point['city'],
                              latitudes=NY_point['lat'],
                              longitudes=NY_point['lng'],
                              radiuses=NY_point['radius']
                             )

In [24]:
LA_venues = getNearbyVenues(names=LA_point['city'],
                              latitudes=LA_point['lat'],
                              longitudes=LA_point['lng'],
                              radiuses=LA_point['radius']
                             )

In [29]:
Seoul_venues = getNearbyVenues(names=Seoul_point['city'],
                              latitudes=Seoul_point['lat'],
                              longitudes=Seoul_point['lng'],
                              radiuses=Seoul_point['radius']
                             )

In [32]:
DC_venues = getNearbyVenues(names=DC_point['city'],
                              latitudes=DC_point['lat'],
                              longitudes=DC_point['lng'],
                              radiuses=DC_point['radius']
                             )

In [37]:
Paris_venues = getNearbyVenues(names=Paris_point['city'],
                              latitudes=Paris_point['lat'],
                              longitudes=Paris_point['lng'],
                              radiuses=Paris_point['radius']
                             )

In [44]:
Durham_venues = getNearbyVenues(names=Durham_point['city'],
                              latitudes=Durham_point['lat'],
                              longitudes=Durham_point['lng'],
                              radiuses=Durham_point['radius']
                             )

In [46]:
Luxembourg_venues = getNearbyVenues(names=Luxembourg_point['city'],
                              latitudes=Luxembourg_point['lat'],
                              longitudes=Luxembourg_point['lng'],
                              radiuses=Luxembourg_point['radius']
                             )

In [10]:
SF_venues = getNearbyVenues(names=SF_point['city'],
                              latitudes=SF_point['lat'],
                              longitudes=SF_point['lng'],
                              radiuses=SF_point['radius']
                             )

In [13]:
Cork_venues = getNearbyVenues(names=Cork_point['city'],
                              latitudes=Cork_point['lat'],
                              longitudes=Cork_point['lng'],
                              radiuses=Cork_point['radius']
                             )

In [26]:
LA_venues.to_csv('LA_venues.csv')

In [22]:
NY_venues.to_csv('NY_venues.csv')

In [31]:
Seoul_venues.to_csv('Seoul_venues.csv')

In [34]:
DC_venues.to_csv('DC_venues.csv')

In [40]:
Paris_venues.to_csv('Paris_venues.csv')

In [49]:
Durham_venues.to_csv('Durham_venues.csv')

In [50]:
Luxembourg_venues.to_csv('Luxembourg_venues.csv')

In [12]:
SF_venues.to_csv('SF_venues.csv')

In [15]:
Cork_venues.to_csv('Cork_venues.csv')

In [3]:
LA_venues = pd.read_csv('LA_venues.csv')
NY_venues = pd.read_csv('NY_venues.csv')
Seoul_venues = pd.read_csv('Seoul_venues.csv')
DC_venues = pd.read_csv('DC_venues.csv')
Paris_venues = pd.read_csv('Paris_venues.csv')
Durham_venues = pd.read_csv('Durham_venues.csv')
Luxembourg_venues = pd.read_csv('Luxembourg_venues.csv')
SF_venues = pd.read_csv('SF_venues.csv')
Cork_venues = pd.read_csv('Cork_venues.csv')

In [4]:
LA_venues.columns

Index(['Unnamed: 0', 'City Name', 'Latitude', 'Longitude', 'Venue',
       'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [57]:
cities_venues = pd.DataFrame(columns=['Unnamed: 0', 'City Name', 'Latitude', 'Longitude', 'Venue',
       'Venue Latitude', 'Venue Longitude', 'Venue Category'])
cities_venues = cities_venues.append(LA_venues, ignore_index=True)
cities_venues = cities_venues.append(NY_venues, ignore_index=True)
cities_venues = cities_venues.append(DC_venues, ignore_index=True)
cities_venues = cities_venues.append(Paris_venues, ignore_index=True)
#cities_venues = cities_venues.append(Durham_venues, ignore_index=True)
#cities_venues = cities_venues.append(Luxembourg_venues, ignore_index=True)
cities_venues = cities_venues.append(SF_venues, ignore_index=True)
#cities_venues = cities_venues.append(Cork_venues, ignore_index=True)

In [58]:
print('There are {} uniques categories.'.format(len(cities_venues['Venue Category'].unique())))

There are 542 uniques categories.


In [59]:
city_onehot = pd.get_dummies(cities_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#city_onehot['Neighborhood'] = city_venues['Neighborhood'] 
city_onehot['Latitude'] = cities_venues['Latitude'] 
city_onehot['Longitude'] = cities_venues['Longitude'] 
# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,Longitude,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,...,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yemeni Restaurant,Yoga Studio,Zoo,Zoo Exhibit,Latitude
0,-118.440230,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33.659541
1,-118.326256,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33.659541
2,-118.326256,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33.659541
3,-118.326256,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33.659541
4,-118.269269,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,33.659541


In [60]:
city_grouped = city_onehot.groupby(['Latitude','Longitude']).mean().reset_index()

In [61]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

city_grouped_clustering = city_grouped.drop(['Latitude','Longitude'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([1, 5, 5, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 7, 7, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       6, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 6, 6, 9, 0, 6, 0, 0, 0, 6, 0, 8, 0, 0, 5, 0, 0,
       0, 2, 0, 0, 2, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

In [62]:
city_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = city_point

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
city_merged = city_merged.join(city_grouped.set_index(['Latitude','Longitude']), on=['lat','lng'])


city_merged.dropna(inplace=True)

In [63]:
city_merged.head()

,city,lat,lng,radius,Cluster Labels,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,...,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yemeni Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,"New York, USA",40.477399,-74.259090,2669,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"New York, USA",40.477399,-74.196989,2669,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"New York, USA",40.477399,-74.134888,2669,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"New York, USA",40.477399,-73.948585,2669,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"New York, USA",40.477399,-73.886484,2669,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
df_coffee = city_merged.loc[:,['city','lat','lng','radius','Cluster Labels','Coffee Shop','Café']]
df_coffee['counter'] = df_coffee['Coffee Shop'] + df_coffee['Café']

In [95]:
df_coffee.groupby(['Cluster Labels']).mean()

,lat,lng,Coffee Shop,Café,counter
Cluster Labels,,,,,
0.0,39.841278,-75.551886,0.024264,0.011565,0.035829
1.0,35.998616,-103.594095,0.000000,0.000000,0.000000
2.0,40.721165,-73.700181,0.000000,0.000000,0.000000
3.0,48.857820,2.303035,0.008209,0.019919,0.028128
4.0,40.501776,-73.855433,0.000000,0.000000,0.000000
5.0,36.359199,-108.190872,0.000000,0.000000,0.000000
6.0,36.060995,-120.495332,0.000000,0.010417,0.010417
8.0,34.337306,-118.155295,0.000000,0.000000,0.000000


In [97]:
df_coffee = df_coffee[(df_coffee['Cluster Labels']==0.0) | (df_coffee['Cluster Labels']==3.0)]

In [102]:
df_coffee['counter'].describe()

count    203.000000
mean       0.034084
std        0.032466
min        0.000000
25%        0.000000
50%        0.029851
75%        0.050000
max        0.160000
Name: counter, dtype: float64

In [103]:
df_coffee = df_coffee[(df_coffee['counter']>0.03) & (df_coffee['counter'] < 0.05)]

In [107]:
df_coffee.to_csv('df_coffee.csv')

# Clustering

# Test Zone

## Task: Change the city with country code to have better geopy result (Done)

## TASK: Calculate the radius parameter(Done)

## TASK: GET VENUES and create feature DataFrame(Done)

## TASK: Clustering(Done)

# Drawing the Map(Working)